In [1]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit, cross_val_score, cross_validate, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
import pickle
import os
pd.pandas.set_option('display.max_columns',None)

In [2]:
%matplotlib inline

In [6]:
import mysql.connector

# Connect to the MySQL database
conn = mysql.connector.connect(
    host='your_host',
    user='your_username',
    password='your_password',
    database='your_database'
)

# Create a SQL query to fetch the data
query = "SELECT * FROM your_table"

# Load the data into a DataFrame
housing_data = pd.read_sql(query, conn)

# Close the database connection
conn.close()


In [9]:
selected_features = list(housing_data.iloc[:, :-1])


### Splitting the Data (Train & Test Sets):

For the purposes of modeling, it will be necessary to split the data into at least training and testing sets. Further splitting may be done for cross validation in the training phase. In order to ensure that the test set is representative of the training set, I'll use stratified sampling to make the test set have the same proportion of price categories defined in the data analysis section (as shown in the pie chart).

In [11]:
'''
Function for standardizing the data, and converting boolean values to numerical.
'''
def standardize_dataframe(df, scaler=None):
    # Convert boolean columns to numerical (0 or 1); and standardize numerical columns
    bool_df = df[[col for col in df.columns if df.dtypes[col] == 'bool']].astype('float64')
    num_df = df[[col for col in df.columns if df.dtypes[col] == 'float64'
                 or df.dtypes[col] == 'int64']]
    # If scaler is given, use it. Otherwise train a new scaler on the numerical columns.
    if(scaler==None):
        scaler = StandardScaler().fit(num_df)
    num_df = pd.DataFrame(scaler.transform(num_df), columns=num_df.columns)
    
    # Ensure that bool_df has ascending indices just like num_df (since scaling resets indices)
    bool_df = bool_df.reset_index().drop(['index'], axis=1)
    
    return num_df.join(bool_df), scaler

In [12]:
# Sampling hyperparameters
test_size = 0.2
intervals = [0, 250000, 400000, 600000, 800000, 1500000, housing_data['latestPrice'].max()]
intervals.sort()
price_categories = pd.cut(housing_data['latestPrice'], intervals)

# Split into training and test sets
split_func = StratifiedShuffleSplit(n_splits=1, test_size=test_size, random_state=46)
indices = list(split_func.split(housing_data, price_categories))[0]
training_data, test_data = housing_data.iloc[indices[0]], housing_data.iloc[indices[1]]
x_train, y_train = training_data[selected_features], training_data['latestPrice']
x_test, y_test = test_data[selected_features], test_data['latestPrice']
y_train, y_test = y_train.reset_index()['latestPrice'], y_test.reset_index()['latestPrice']

# Standardize the sets (using only parameters from training set)
x_train, scaler = standardize_dataframe(x_train)
x_test, _ = standardize_dataframe(x_test, scaler)
# Full sets standardized
x, y = pd.concat([x_train, x_test], ignore_index=True), pd.concat([y_train, y_test], ignore_index=True)

Just to be sure that the test data is in fact representative of the training data, we can compare the price labels in each set using a scatter plot. The plot below shows that the training labels (in blue) and the test labels (in red) overlap pretty well, and that each includes a good sample of outliers from the data. The pie charts to the left show the true proportions of price categories in each set. We can see that they are nearly identical, and thus the test set is in fact representative of the training set.

# Modeling:
In this section I'll be creating models to predict the price of a home based on its other features.

The cell below defines several functions for saving and loading models.

In [19]:
model_dir = '/home/yadhu_sudhan/MEGA/PROJECT/SUYATI/SUYATI PROJECT/models/'
linear_regressor_name = "linear_regressor"
decision_tree_regressor_name = "decision_tree_regressor"
random_forest_regressor_name = "random_forest_regressor"

def save_sklearn_model(model, model_name):
    with open(model_dir + model_name + '.pkl', 'wb') as file:  pickle.dump(model, file)

def load_sklearn_model(model_name):
    with open(model_dir + model_name + '.pkl', 'rb') as file:
        model = pickle.load(file)
    return model

In [22]:
model = RandomForestRegressor()
#best_params = {'n_estimators': 100, 'max_features': 4}
#model = RandomForestRegressor(n_estimators=best_params['n_estimators'], max_features=best_params['max_features'])
model = RandomForestRegressor(n_estimators=100, max_features=4)
model.fit(x_train, y_train)


y_pred = model.predict(x_test)
r2 = r2_score(y_test, y_pred)
adjusted_r2 = 1 - (1-r2)*(len(y)-1)/(len(y)-x.shape[1]-1)
print("Best Model R-squared:\t", np.around(r2, 2))
print("Best Model Adjusted R-squared:\t", np.around(adjusted_r2, 2))

# Save model
save_sklearn_model(model, random_forest_regressor_name)
print("\nSaved Model Location:\t" + model_dir + random_forest_regressor_name + '.pkl')



Best Model R-squared:	 0.52
Best Model Adjusted R-squared:	 0.52

Saved Model Location:	/home/yadhu_sudhan/MEGA/PROJECT/SUYATI/SUYATI PROJECT/models/random_forest_regressor.pkl


In [30]:
feature_names = ['livingAreaSqFt','numOfBathrooms','lotSizeSqFt','numOfBedrooms','numOfStories','numOfPhotos','numOfPatioAndPorchFeatures','numOfParkingFeatures','latest_saleyear','numOfSecurityFeatures','hasSpa','hasView']

In [31]:
y_pred = model.predict(housing_data[feature_names])

In [ ]:
import mysql.connector
import pandas as pd

# MySQL database connection details
host = 'your_host'
user = 'your_username'
password = 'your_password'
database = 'your_database'

# Table name for predictions
table_name = 'predictions_table'

# Convert predictions to a DataFrame
predictions_df = pd.DataFrame({'Prediction': y_pred})

# Connect to the MySQL database
conn = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database=database
)

# Create a cursor
cursor = conn.cursor()

# Create the predictions table if it doesn't exist
create_table_query = f"CREATE TABLE IF NOT EXISTS {table_name} (id INT AUTO_INCREMENT PRIMARY KEY, Prediction FLOAT)"
cursor.execute(create_table_query)

# Insert the predictions into the database
insert_query = f"INSERT INTO {table_name} (Prediction) VALUES (%s)"
for index, row in predictions_df.iterrows():
    cursor.execute(insert_query, (row['Prediction'],))

# Commit the changes and close the cursor and connection
conn.commit()
cursor.close()
conn.close()
